# python实现HTML生成PDF

### 生成pdf报告需求

1. 动态生成html
2. 生成的html字符串转pdf文件
3. 报告具有封面,封底,目录,书签

## 安装依赖

下载并系统安装：wkhtmltopdf, 链接：https://wkhtmltopdf.org/index.html,

注意: python 64位的对应wkhtmltopdf 64位版本.

总共需要安装：

1. pdfkit
2. wkhtmltopdf(python包)
3. wkhtmltopdf(windows安装包)以及路径配置 

流程: 程序会使用pdfkit，pdfkit会调用wkhtmltopdf，而wkhtmltopdf会调用windows中的wkhtmltopdf.exe来转化html为pdf。

### windows

将wkhtmltopdf的bin目录添加至path环境变量，注意：重启生效。命令行输入：

In [ ]:
SETX PATH "%PATH%;C:\\Program Files\\wkhtmltopdf\\bin" /M

临时导入环境变量，命令行输入：

In [ ]:
set PATH=C:\Program Files\wkhtmltopdf\bin;%PATH%

### Linux

In [ ]:
sudo apt-get install wkhtmltopdf

### 安装python第三方库：

In [ ]:
pip install pdfkit
pip install wkhtmltopdf

### pdfkit生成PDF的一些缺陷

1. 只能生成封面,不能生成封底
2. 目录生成在最后,在正文的后面,而不是正文的前面
3. 生成正文,目录和封面后都会产生一个空白页
4. 封面只能是html文件,不支持字符串的html
5. 在css中设置了page的大小为A4的大小,实际生成的封面要小一些,不知为何
6. 生成封面和封底会产生部分margin,通过生成html,在chrome控制台观测元素发现body默认带有8px的margin,在css中设置body的margin为0即可解决

## 生成HTML

安装jinja2，并使用jinja2动态生成HTML模版 

In [ ]:
pip install jinja2

构造HTML模版，这个根据具体需求并参考jinja2构建模版方法，自行构建。

### html转pdf过程中遇到的若干问题

### 1. 封面要动态生成,而pdfkit不支持html字符串,必须是html文件

由于wkhtmltopdf要求封面和目录单独指定选项,且封面选项是一个封面的html文件路径,这就导致无法生成动态封面

解决方法:
1. 动态生成封面的字符串写入到html文件中,然后再指定这个封面html文件路径到封面选项
2. 封面单独作为一个pdf生成

由于wkhtmltopdf不能生成封底,(没有这个选项),因此选择各自生成单独生成封面和封底的pdf,然后再组装成一个pdf

### 2. 单独生成封面封底的pdf出现空白的margin

这毫无疑问肯定是样式导致的margin, 将动态生成的封面和封底的html字符串写入到html文件中,然后在chrome浏览器中打开,排查发现html中body标签含有8px的margin

解决方法: 在css中设置body样式的margin为0, 注意: css的语法形式

```
body {
    margin: 0 0 0 0;
    color: #333333;
    font: 16px/1.5 BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Helvetica, "PingFang SC", "Hiragino Sans GB", "Microsoft YaHei", SimSun, sans-serif;
}
```

### 3.生成的目录在正文后面

使用pdfkit生成的pdf后发现目录在正文后面,这不知道是wkhtmltopdf工具的问题还是pdfkit的问题

解决办法: 使用pypdf2这个库,对生成的pdf进行分割然后和封面和封底一起合并

**注意:** 使用pypdf2这个库分割合并pdf后,会导致原来pdf的标签和链接(link)全部失效

### 4. 生成pdf分页问题

html生成pdf后发现pdf页面错乱

解决办法: 在css设置分页属性对html设置分页,在css样式中需要分页的标签设置:`page-break-inside: avoid !important;`属性,则在html转pdf时在超过一页的内容的地方自动转到下一页上去.

```
table, tr, td, th, tbody, thead, tfoot {
    page-break-inside: avoid !important;
}

div {
    page-break-inside: avoid !important;
}
```

### 5. 书签重新生成

由于使用pypdf2分割合并pdf后,原始pdf的书签和链接全部丢失了

解决办法: 使用pypdf2重新生成书签,包括二级,三级书签的生成

设置wkhtmltopdf的`dump-outline`选项,在html转pdf时生成outline.html到一个指定路径中去,生成的outline.html是一个xml文件,然后通过beautifulsoup库读取这个outline.html文件,将outline结构解析出来,用来生成多级书签
```
'dump-outline': os.path.join(STATIC_PATH, 'outline.html'),
```

### 6. 如何使用pypdf2生成多级书签

pypdf2中没有关于生成多级书签的说明,可能是我在官方文档中没有找到,后来在github看到了一个人用pypdf2生成了多级标签,源码如下:
```
def addBookmarks(pdf_in_filename, bookmarks_tree, pdf_out_filename=None):
    """Add bookmarks to existing PDF files
    Home:
        https://github.com/RussellLuo/pdfbookmarker
    Some useful references:
        [1] http://pybrary.net/pyPdf/
        [2] http://stackoverflow.com/questions/18855907/adding-bookmarks-using-pypdf2
        [3] http://stackoverflow.com/questions/3009935/looking-for-a-good-python-tree-data-structure
    """
    pdf_out = PdfFileMerger()

    # read `pdf_in` into `pdf_out`, using PyPDF2.PdfFileMerger()
    with open(pdf_in_filename, 'rb') as inputStream:
        pdf_out.append(inputStream, import_bookmarks=False)

    # copy/preserve existing metainfo
    pdf_in = PdfFileReader(pdf_in_filename)
    metaInfo = pdf_in.getDocumentInfo()
    if metaInfo:
        pdf_out.addMetadata(metaInfo)

    def crawl_tree(tree, parent):
        for title, pagenum, subtree in tree:
            current = pdf_out.addBookmark(title, pagenum, parent)  # add parent bookmark
            if subtree:
                crawl_tree(subtree, current)

    # add bookmarks into `pdf_out` by crawling `bookmarks_tree`
    crawl_tree(bookmarks_tree, None)

    # get `pdf_out_filename` if it's not specified
    if not pdf_out_filename:
        name_parts = os.path.splitext(pdf_in_filename)
        pdf_out_filename = name_parts[0] + '-new' + name_parts[1]

    # wrie `pdf_out`
    with open(pdf_out_filename, 'wb') as outputStream:
        pdf_out.write(outputStream)

```  
`addBookmark`第3个参数parent接受一个父书签对象,默认为None,即根书签
原来`addBookmark`函数会返回一个当前的书签对象,如果想要设置当前书签的自书签,则以当前书签作为parent传入`addBookmark`的第三个参数,

更好的方法(这里只生成两级书签):
```
import os
from app.constants import STATIC_PATH
from PyPDF2 import PdfFileWriter, PdfFileReader


class Pdf(object):
    def __init__(self, path):
        self.path = path
        self.reader = PdfFileReader(open(path, "rb"))
        self.writer = PdfFileWriter()
        self.writer.appendPagesFromReader(self.reader)
        self.writer.addMetadata(self.reader.getDocumentInfo())

    @property
    def new_path(self):
        name, ext = os.path.splitext(self.path)
        return name + '_new' + ext

    def add_bookmark(self, title, pagenum, parent=None):
        return self.writer.addBookmark(title, pagenum, parent=parent)

    def save_pdf(self):
        print(self.new_path)
        with open(self.new_path, 'wb') as out:
            self.writer.write(out)

if __name__ == '__main__':
    pdf_path = os.path.join(STATIC_PATH, 'report.pdf')
    pdf = Pdf(pdf_path)
    bookmarks = [{'title': 'cover', 'num': 1, 'parent': None},
                 {'title': '一、检测报告', 'num': 2, 'parent': None},
                 {'title': '二、监控列表', 'num': 3, 'parent': None},
                 {'title': '1、监控统计', 'num': 4, 'parent': 3},
                 {'title': '2、监控列表', 'num': 5, 'parent': 3},
                 {'title': '三、事件报告', 'num': 10, 'parent': None},
                 {'title': '1、用户异常登录', 'num': 12, 'parent': 10},
                 {'title': '2、Webshell查杀', 'num': 15, 'parent': 10},
                 {'title': '3、关键文件改动', 'num': 22, 'parent': 10},
                 {'title': '4、恶意进程告警', 'num': 29, 'parent': 10},
                 {'title': '5、反弹Shell检测', 'num': 200, 'parent': 10},
                 {'title': '6、可疑连接', 'num': 210, 'parent': 10},
                 {'title': '四、漏洞报告', 'num': 212, 'parent': None},
                 {'title': '五、安全基线', 'num': 220, 'parent': None}]
    current = None
    for item in bookmarks:
        if item['parent'] is None:
            current = pdf.add_bookmark(item['title'], item['num'], parent=None)
        else:
            pdf.add_bookmark(item['title'], item['num'], parent=current)
    pdf.save_pdf()
```    

### 7. 如何创建目录链接(link)

暂未解决

### python中对应参数设置

* 在python中所有参数移除选项名字前面的 '--' ;
* 如果选项没有值, 使用None, False 或 ''(空字符) 作为选项的值
* 对于多种选择, 在多个值的时候你可能会用到列表或者元组进行存储 (列入自定义头文件授权信息) 你需要两个元组存放 (看看以下例子).
```
'custom-header' : [
        ('Accept-Encoding', 'gzip')
    ]
'cookie': [
    ('cookie-name1', 'cookie-value1'),
    ('cookie-name2', 'cookie-value2'),
]
```
* 默认情况下, PDFKit 将会显示所有的 wkhtmltopdf 输出. 如果不想看到这些信息，你需要传递一个 quiet 选项: `quiet: ''`
* 由于wkhtmltopdf的命令语法 , TOC 和 Cover 选项必须分开指定:

```
toc = {
 'xsl-style-sheet': 'toc.xsl'
 } 
cover = 'cover.html' 
pdfkit.from_file('file.html', options=options, toc=toc, cover=cover)
```

* 当我们转换文件、或字符串的时候，可以通过css选项指定扩展的 CSS 文件。

```
# 单个 CSS 文件 
css = 'example.css' 
pdfkit.from_file('file.html', options=options, css=css) 
# 多个 CSS files 
css = ['example.css', 'example2.css'] 
pdfkit.from_file('file.html', options=options, css=css)

```

* 通过HTML中的meta tags传递任意选项：

```
body = """
        <html>
          <head>
            <meta name="pdfkit-page-size" content="Legal"/>
            <meta name="pdfkit-orientation" content="Landscape"/>
          </head>
          Hello World!
          </html>
        """ 
#with --page-size=Legal and --orientation=Landscape
pdfkit.from_string(body, 'out.pdf') 

```

### 经常使用的wkhtmltopdf的选项

```
# COVER_OPTIONS
COVER_OPTIONS = {
    'quiet': '',
    'page-size': 'A4',
    'margin-top': '0in',
    'margin-right': '0in',
    'margin-bottom': '0in',
    'margin-left': '0in',
    'encoding': 'UTF-8'
}
# page options
PAGE_OPTIONS = {
    'quiet': '',
    'page-size': 'A4',
    'dump-outline': os.path.join(STATIC_PATH, 'outline.html'),
    # 'no-outline': None,
    'margin-top': '1.0in',
    'margin-bottom': '1.0in',
    'margin-left': '0.75in',
    'margin-right': '0.75in',

    'encoding': 'UTF-8',
    'header-line': '-',
    'header-font-size': 10,
    'header-spacing': 5,
    'header-left': 'Life is short',
    'header-right': 'You need python!',
    'footer-spacing': 5,
    'footer-center': "[page] / [toPage]",
    'footer-line': '-'
}
# table of content
TOC = {
    'toc-header-text': '目 录',
    # 第级标题在目录中的缩进宽度(默认为1em)
    'toc-level-indentation': '0.5in',
}
```

## 使用python动态生成HTML

当生成一个具有格式要求的报告时,且报告的内容是动态变化的,该如何生成这个动态内容的报告呢

报告内容是动态变化的,但是报告的格式是固定的,因此选择生成html报告是一个非常合适的选择:
* html做报告格式的骨架
* CSS做报告各种样式要求的设置

剩下的问题就是怎么组织这个html骨架模板了,自然而然就想到著名的jiaja2库,可以将html分层有组织的组合在一起.

下面展示一个html组织架构

```html
<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <title>检测报告</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href={{ CSS }}>
    <!-- <link rel="stylesheet" type="text/css" media="screen" href={{ css_path }} /> -->
</head>

<body>
    <!-- 报告主要内容 begin -->
    <div class="report-container">
        <div class="report-content">
            <div class="per-data">
                {% include 'seg1.html' %}
            </div>
            <div class="per-data">
                {% include 'seg2.html' %}
            </div>
            <div class="per-data">
                {% include 'seg3.html' %}
            </div>
            <div class="per-data">
                {% include 'seg4.html' %}
            </div>
            <div class="per-data">
                {% include 'seg5.html' %}
            </div>
        </div>
    </div>
    <!-- 报告主要内容 end -->
</body>
</html>

```

### 遇到的一些问题

#### 关于no such file or directory:b'' 这种错误

1. 在python中出现时，意味着有.exe文件需要被调用，而该.exe文件没有被安装或者在环境变量中没有添加该.exe的路径。
2. 有时候需要改pdfkit代码为下列两句，才可消除错误：

In [ ]:
config=pdfkit.configuration(wkhtmltopdf=r"C:\Program File\wkhtmltopdf\bin\wkhtmltopdf.exe")
pdfkit.from_url(url, name,configuration=config)

### 用浏览器打开jinja2模版

In [ ]:
import os
import logging
import webbrowser
from jinja2 import Environment
from jinja2 import FileSystemLoader
from jinja2 import select_autoescape
from traceback import format_exc


from app.logger import create_logger

templates_path = 'app/templates'
env = Environment(loader=FileSystemLoader(templates_path),
                  autoescape=select_autoescape(['html', 'xml']))

print(env.list_templates())


def get_report_html(template_name):
    try:
        if template_name in env.list_templates():
            template = env.get_template(template_name)
            css_path = os.path.abspath('app/static/css/style.css')
            html = template.render({'css_path': css_path})
            return html
        else:
            logging.error('there is no {} template existed'.format(template_name))
    except Exception as e:
        logging.error(format_exc())


def open_webbrowser(html):
    try:
        with open('dst.html', 'w', encoding='utf-8') as f:
            f.write(html)

        html_url = 'file:///{}'.format(os.path.abspath('dst.html'))
        webbrowser.open(html_url)
    except Exception as e:
        logging.error(e)


if __name__ == '__main__':
    create_logger(log_name='rendtemplate.log')
    template_name = 'reportmain.html'
    html = get_report_html(template_name)
    open_webbrowser(html)